In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.utils import data
from d2l import torch as d2l
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm
import numpy as np

In [14]:
T = 2000
time = torch.arange(1, T + 1, dtype=torch.float32)
# x = torch.sin(0.01 * time)
x = torch.sin(0.01 * time) + torch.normal(0, 0.02, (T,))

In [15]:
px.scatter(x=time, y=x)

In [16]:
tau = 10
features = torch.zeros((T - tau, tau))

for i in range(tau):
    features[:, i] = x[i: T - tau + i]

labels = x[tau: ].reshape((-1, 1))

In [17]:
dataset = data.TensorDataset(features, labels)
train_iter = data.DataLoader(dataset=dataset, batch_size=32, shuffle=True)

In [18]:
net = nn.Sequential(
    nn.Linear(tau, 64),
    nn.ReLU(),
    nn.Linear(64, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 1)
)

def init_weight(layer):
    if type(layer) == nn.Linear:
        nn.init.xavier_uniform_(layer.weight)
        
net.apply(init_weight)

Sequential(
  (0): Linear(in_features=10, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=1, bias=True)
)

In [19]:
trainer = torch.optim.Adam(net.parameters(), lr=0.05)
loss = nn.MSELoss(reduction='none')

for epoch in range(20):
    for X, y in tqdm(train_iter):
        trainer.zero_grad()
        pred = net(X)
        l = loss(pred, y)
        l.sum().backward()
        trainer.step()

100%|██████████| 63/63 [00:00<00:00, 1001.11it/s]


In [20]:
onestep_pred = net(features)

In [21]:
fig = go.Figure()
fig.add_traces([
    go.Scatter(x=time[tau: ], y=labels.flatten(), mode='markers'),
    go.Scatter(x=time[tau: ], y=onestep_pred.flatten().detach().numpy(), mode='markers')
])

In [22]:
multistep_preds = torch.zeros((T))
multistep_preds[0: tau] = x[0: tau]

In [23]:
multistep_preds.shape

torch.Size([2000])

In [25]:
for i in range(len(multistep_preds) - tau):
    print(i, end='\r')
    pred = net(multistep_preds[i: tau + i].reshape((1, -1))).detach().flatten()[0]
    multistep_preds[i + tau] = pred

In [26]:
fig = go.Figure()
fig.add_traces([
    go.Scatter(x=time[tau: ], y=labels.flatten(), mode='markers'),
    go.Scatter(x=time[tau: ], y=multistep_preds.flatten().detach().numpy(), mode='markers')
])